# **1. Import Libraries**

In [1]:
!pip install tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
import numpy as np


# **2. Load and Preprocess Dataw Section**

In [3]:
# Load transcriptions
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = file.read()
    return data

# Assuming all transcriptions are stored in a single text file
data = load_data('/content/cleaned_transcripts.txt')

# Preprocess the data
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in data.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = tf.keras.utils.to_categorical(label, num_classes=total_words)


# **3. Build the Modelction**

In [4]:
model = Sequential([
    Embedding(total_words, 100, input_length=max_sequence_len-1),
    LSTM(150, return_sequences=True),
    Dropout(0.2),
    LSTM(100),
    Dense(total_words, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 99, 100)           362500    
                                                                 
 lstm (LSTM)                 (None, 99, 150)           150600    
                                                                 
 dropout (Dropout)           (None, 99, 150)           0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               100400    
                                                                 
 dense (Dense)               (None, 3625)              366125    
                                                                 
Total params: 979625 (3.74 MB)
Trainable params: 979625 (3.74 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# **4. Train the Model**

In [5]:
history = model.fit(predictors, label, epochs=50, verbose=1)


Epoch 1/50
3366/3366 [==============================] - 79s 22ms/step - loss: 5.6707 - accuracy: 0.0700
Epoch 2/50
3366/3366 [==============================] - 49s 14ms/step - loss: 4.9960 - accuracy: 0.1212
Epoch 3/50
3366/3366 [==============================] - 48s 14ms/step - loss: 4.7783 - accuracy: 0.1380
Epoch 4/50
3366/3366 [==============================] - 54s 16ms/step - loss: 4.3842 - accuracy: 0.1755
Epoch 5/50
3366/3366 [==============================] - 52s 16ms/step - loss: 4.0393 - accuracy: 0.2150
Epoch 6/50
3366/3366 [==============================] - 47s 14ms/step - loss: 3.7699 - accuracy: 0.2525
Epoch 7/50
3366/3366 [==============================] - 49s 14ms/step - loss: 3.5591 - accuracy: 0.2829
Epoch 8/50
3366/3366 [==============================] - 50s 15ms/step - loss: 3.3854 - accuracy: 0.3097
Epoch 9/50
3366/3366 [==============================] - 49s 15ms/step - loss: 3.2430 - accuracy: 0.3319
Epoch 10/50
3366/3366 [==============================] - 47s 14m

# **5. Generate Text**

In [6]:
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

print(generate_text("I feel", 50, max_sequence_len))


I feel like my whole life is going to be spent standing on the beach with my eyes wide open and my hands clasped expectantly waiting for fish to show up and the fish won't show up and the fish will go to the beach and we took off our shoes and
